In [10]:
import sys
print(sys.version_info)

#!{sys.executable} -m pip install dwave-ocean-sdk
#!{sys.executable} -m pip install dwave-neal
#!{sys.executable} -m pip install dwave-system
#!{sys.executable} -m pip install dwave-inspector

import neal

sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
  Using cached dwave_inspector-0.3.0-py3-none-any.whl (26 kB)
  Using cached Flask-2.2.2-py3-none-any.whl (101 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
  Using cached importlib_metadata-5.0.0-py3-none-any.whl (21 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
%load_ext autoreload
%autoreload

import getpass
import common
import maxcut
import dwave.inspector
import csv
from itertools import count

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Constants

In [12]:
SEPERATOR = "--------------------------------------------------"
path = "instance-final/mc_320_003_000.txt"
outputFile = "maxcut_results.csv"

In [13]:
# Get key to use for D-Wave Call
prompt = "" 
token = ""
try:
    prompt = getpass.getpass(prompt="D-Wave Key: ", stream=None) 
except Exception as error:
    print('ERROR', error)
else:
    print("Key Accepted")

D-Wave Key: ········
Key Accepted


In [16]:
# Load nodes and edges
nodes, edges = common.read_maxcut_instance(path)

# Simulated Annealing

In [17]:
result = maxcut.maxcut_sa(nodes, edges, 5000)
print(result)

{'stats': {'max': 441, 'mean': 438.4856, 'sd': 1.4621286550918864, 'wallClockStart': 1667457279.6667407, 'wallClockEnd': 1667457295.6176667}, 'timing': {'beta_range': [0.23104906018664842, 4.605170185988092], 'beta_schedule_type': 'geometric'}, 'solution': {0: 1, 1: -1, 2: 1, 3: -1, 4: 1, 5: 1, 6: 1, 7: -1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: -1, 14: -1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: -1, 21: -1, 22: 1, 23: 1, 24: -1, 25: -1, 26: -1, 27: 1, 28: -1, 29: -1, 30: 1, 31: 1, 32: 1, 33: 1, 34: -1, 35: 1, 36: -1, 37: -1, 38: -1, 39: -1, 40: -1, 41: -1, 42: -1, 43: 1, 44: -1, 45: -1, 46: -1, 47: -1, 48: -1, 49: 1, 50: 1, 51: 1, 52: -1, 53: -1, 54: 1, 55: -1, 56: 1, 57: 1, 58: 1, 59: 1, 60: -1, 61: -1, 62: 1, 63: -1, 64: 1, 65: -1, 66: -1, 67: 1, 68: -1, 69: -1, 70: 1, 71: 1, 72: -1, 73: 1, 74: 1, 75: -1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: -1, 83: 1, 84: 1, 85: -1, 86: -1, 87: -1, 88: -1, 89: 1, 90: -1, 91: -1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: -1, 98: 1, 99: -1, 100: -

# Quantum Annealing

In [18]:
embedding = None
shots = 100

with open(outputFile, "w") as f:

     optimal = common.optimal_maxcut(path)

     writer = csv.writer(f)
     writer.writerow(["file", "shots", "qpu_anneal_time_per_sample", "max", "optimal", "mean", "sd",
          "qpu_sampling_time", "qpu_readout_time_per_sample", "qpu_access_time", \
          "qpu_access_overhead_time", "qpu_programming_time", "qpu_delay_time_per_sample", "qpu_programming_time", \
          "total_post_processing_time", "post_processing_overhead_time", "wallclockStart", "wallclockEnd", "embedding", "wallTime"])

     for i in range(8):
          x = 2**i
          if x > 200:
               break

          embeddingCreated = True if embedding==None else False
          print ("Created " if embeddingCreated else "Reused ", "Embedding")

          sampleset, result, embedding = maxcut.maxcut_qa(nodes, edges, shots, token, x, embedding)

          # sample to file
          timing = sampleset.info["timing"]
          data = [path, shots, timing["qpu_anneal_time_per_sample"], result["stats"]["max"], optimal, result["stats"]["mean"], result["stats"]["sd"],
               timing["qpu_sampling_time"], timing["qpu_readout_time_per_sample"], timing["qpu_access_time"], \
               timing["qpu_access_overhead_time"], timing["qpu_programming_time"], timing["qpu_delay_time_per_sample"], timing["qpu_programming_time"], \
               timing["total_post_processing_time"], timing["post_processing_overhead_time"], result["stats"]["wallClockStart"], result["stats"]["wallClockEnd"], \
               "Created" if embeddingCreated else "Reused", result["stats"]["wallClockEnd"] - result["stats"]["wallClockStart"] ]

          writer.writerow(data)

Created  Embedding


ValueError: API token not defined

In [19]:
print(result["stats"])

print(sampleset.info)
print(SEPERATOR)
timing = sampleset.info["timing"]
data = [timing["qpu_anneal_time_per_sample"], result["stats"]["max"], result["stats"]["mean"], result["stats"]["sd"],
    timing["qpu_sampling_time"], timing["qpu_readout_time_per_sample"], timing["qpu_access_time"], \
    timing["qpu_access_overhead_time"], timing["qpu_programming_time"], timing["qpu_delay_time_per_sample"], timing["qpu_programming_time"], \
    timing["total_post_processing_time"], timing["post_processing_overhead_time"]]

print (data)

{'max': 441, 'mean': 438.4856, 'sd': 1.4621286550918864, 'wallClockStart': 1667457279.6667407, 'wallClockEnd': 1667457295.6176667}


NameError: name 'sampleset' is not defined

In [20]:
sampleset, result, embedding = maxcut.maxcut_qa(nodes, edges, 5000, token, 1, None)

ValueError: API token not defined

In [21]:
sampleset, result, a = maxcut.maxcut_qa(nodes, edges, 5000, token, 16, embedding)

# sample to file
timing = sampleset.info["timing"]
data = [path, shots, timing["qpu_anneal_time_per_sample"], result["stats"]["max"], optimal, result["stats"]["mean"], result["stats"]["sd"],
    timing["qpu_sampling_time"], timing["qpu_readout_time_per_sample"], timing["qpu_access_time"], \
    timing["qpu_access_overhead_time"], timing["qpu_programming_time"], timing["qpu_delay_time_per_sample"], timing["qpu_programming_time"], \
    timing["total_post_processing_time"], timing["post_processing_overhead_time"], result["stats"]["wallClockStart"], result["stats"]["wallClockEnd"], \
    "Created" if embeddingCreated else "Reused", result["stats"]["wallClockEnd"] - result["stats"]["wallClockStart"] ]

writer.writerow(data)

ValueError: API token not defined